<a href="https://colab.research.google.com/github/ryann-arruda/deep_learning_algorithms/blob/main/perceptron/pytorch_implementation/multiclass/perceptron_multiclass_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn.functional as F

## **Loading Dataset**

In [2]:
data = load_iris()

In [3]:
x = data.data

In [4]:
y = data.target

## **Data Normalization**

In [5]:
x = (x - x.min(axis=0))/(x.max(axis=0) - x.min(axis=0))

## **Separating the Dataset**

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=27)

## **Creating the Dataset Object**

In [7]:
class SpecificDataset(Dataset):

  def __init__(self, x, y):
    self.features = torch.tensor(x, dtype=torch.float32)
    self.labels = torch.tensor(y, dtype=torch.float32)

  def __getitem__(self, index):
    x = self.features[index]
    y = self.labels[index]

    return x, y

  def __len__(self):
    return self.labels.shape[0]

In [8]:
train_dataset = SpecificDataset(x_train, y_train)
test_dataset = SpecificDataset(x_test, y_test)

In [9]:
train_dataloader = DataLoader(dataset=train_dataset,
                              batch_size=5,
                              shuffle=True)

test_dataloader = DataLoader(dataset=test_dataset,
                             batch_size=5,
                             shuffle=False)

## **Perceptron**

In [31]:
class Perceptron(torch.nn.Module):

  def __init__(self, num_features):
    super().__init__()
    self.linear = torch.nn.Linear(num_features, 3)

  def forward(self, x):
    return self.linear(x)

In [43]:
model = Perceptron(4)

In [44]:
optimizer = torch.optim.SGD(model.parameters(), lr=1.0)

## **Training**

In [45]:
epochs = 10

In [46]:
for epoch in range(epochs):
  model.train()

  for index, (x, y) in enumerate(train_dataloader):
    z = model(x)
    y = y.long()
    loss = F.cross_entropy(z, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f'Epoch: {epoch + 1:03d}/{epochs:03d} | Batch: {index + 1:03d}/{len(train_dataloader):03d} | Loss: {loss:.3f}')

Epoch: 001/010 | Batch: 001/021 | Loss: 1.277
Epoch: 001/010 | Batch: 002/021 | Loss: 1.320
Epoch: 001/010 | Batch: 003/021 | Loss: 1.150
Epoch: 001/010 | Batch: 004/021 | Loss: 0.909
Epoch: 001/010 | Batch: 005/021 | Loss: 0.923
Epoch: 001/010 | Batch: 006/021 | Loss: 1.077
Epoch: 001/010 | Batch: 007/021 | Loss: 0.667
Epoch: 001/010 | Batch: 008/021 | Loss: 0.917
Epoch: 001/010 | Batch: 009/021 | Loss: 0.842
Epoch: 001/010 | Batch: 010/021 | Loss: 0.913
Epoch: 001/010 | Batch: 011/021 | Loss: 0.743
Epoch: 001/010 | Batch: 012/021 | Loss: 0.338
Epoch: 001/010 | Batch: 013/021 | Loss: 0.963
Epoch: 001/010 | Batch: 014/021 | Loss: 0.801
Epoch: 001/010 | Batch: 015/021 | Loss: 0.761
Epoch: 001/010 | Batch: 016/021 | Loss: 0.709
Epoch: 001/010 | Batch: 017/021 | Loss: 0.401
Epoch: 001/010 | Batch: 018/021 | Loss: 0.340
Epoch: 001/010 | Batch: 019/021 | Loss: 1.036
Epoch: 001/010 | Batch: 020/021 | Loss: 0.822
Epoch: 001/010 | Batch: 021/021 | Loss: 0.564
Epoch: 002/010 | Batch: 001/021 | 

## **Evaluating the Results**

In [47]:
def accuracy(model, test_dataloader):
  accuracy = 0.0
  total = 0

  model.eval()
  for index, (x, y) in enumerate(test_dataloader):

    with torch.inference_mode():
      z = model(x)

    labels = torch.argmax(z, dim=1)

    labels = labels.view(y.shape).to(y.dtype)

    results = labels == y
    accuracy += torch.sum(results).item()
    total += len(results)

  return accuracy/total

In [48]:
print(f'Accuracy (train): {accuracy(model, train_dataloader):.2f}')

Accuracy (train): 0.94


In [49]:
print(f'Accuracy (test): {accuracy(model, test_dataloader):.2f}')

Accuracy (test): 0.89
